In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
import scipy as sp
import scipy.stats as stats
%matplotlib inline

In [2]:
# pricing parameters

# economics
K = 100.0
T = 1.

# market data
Vol = 0.50
R = 0.05
S0 = 90.0

In [3]:
# black scholes analytic solutions

def d1(s, k, r, vol, t):
    return (sp.log(s/k)+(r+(0.5*vol*vol))*t)/(vol*sp.sqrt(t))
def d2(s, k, r, vol, t):
    return (sp.log(s/k)+(r-(0.5*vol*vol))*t)/(vol*sp.sqrt(t))
def bs_call(s, k, r, vol, t):
    return s*stats.norm.cdf(d1(s, k, r, vol, t)) - k*sp.exp(-1.*r*t)*stats.norm.cdf(d2(s, k, r, vol, t))
def bs_put(s, k, r, vol, t):
    return -1.*s*stats.norm.cdf(-1.*d1(s, k, r, vol, t)) + k*sp.exp(-1.*r*t)*stats.norm.cdf(-1.*d2(s, k, r, vol, t))
def bs_digital_call(s, k, r, vol, t):
    return sp.exp(-1.*r*t)*stats.norm.cdf(d2(s, k, r, vol, t))
def bs_digital_put(s, k, r, vol, t):
    return sp.exp(-1.*r*t)*(1.-stats.norm.cdf(d2(s, k, r, vol, t)))
def zcb(r ,t):
    return sp.exp(-1.*r*t)

In [4]:
# payoff implementations

class call_payoff:
    def __init__(self, k):
        self.k = k
    def calc(self, spots):
        return np.maximum(spots-self.k, 0.)

class digital_call_payoff:
    def __init__(self, k):
        self.k = k
    def calc(self, spots):
        return 1. * (spots > self.k)

In [5]:
# tree product implementations

class european_tree_product:
    def __init__(self, final_time, payoff):
        self.final_time = final_time
        self.payoff = payoff
    def final_time(self):
        return self.final_time
    def final_payoff(self, spot):
        return self.payoff.calc(spot)
    def pre_final_value(self, spot, time, discounted_future_value):
        return discounted_future_value
    
class american_tree_product:
    def __init__(self, final_time, payoff):
        self.final_time = final_time
        self.payoff = payoff
    def final_time(self):
        return self.final_time
    def final_payoff(self, spot):
        return payoff.calc(spot)
    def pre_final_value(self, spot, time, discounted_future_value):
        return max(payoff.calc(spot), discounted_future_value)
    

In [12]:
# binomial tree

class simple_binomial_tree:
    def __init__(self, s0, r, vol, num_of_steps, time):
        self.spot_tree = []
        log_spot = sp.log(s0)
        for i in range(num_of_steps):
            t = (i*time)/num_of_steps
            moved_log_spot = log_spot + (r*t)
            moved_log_spot -= 0.5*vol*vol*t
            sd = vol*sp.sqrt(time/num_of_steps)
            self.spot_tree.append([sp.exp(moved_log_spot+(s*sd)) for s in range(-i, i, 2)])
        self.df = sp.exp(-1.*r*time/num_of_steps)
        

In [13]:
sbt = simple_binomial_tree(S0, R, Vol, 10, 1.0)